In [11]:
import unittest
import re

In [9]:
test_prog = '''b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10'''.split('\n')
real_prog = [i.strip() for i in open('input.txt').readlines()]

In [10]:
real_prog

['ioe dec 890 if qk > -10',
 'gif inc -533 if qt <= 7',
 'itw dec 894 if t != 0',
 'nwe inc 486 if hfh < -2',
 'xly inc 616 if js >= -3',
 'j inc 396 if b != -5',
 'nwe dec -637 if uoc > 0',
 'b inc 869 if yg >= -3',
 'gif dec -221 if iyu < 0',
 'tc dec -508 if gy >= -7',
 'x dec 637 if gif < -526',
 'nwe dec -185 if nwe != -8',
 'x inc 638 if b != 869',
 'ih dec -722 if itw > 9',
 'xly inc -38 if ih >= 8',
 'hm dec 910 if t == 0',
 'uoc dec -585 if qt == 0',
 'js dec -325 if hm == -910',
 'yr dec -922 if cp != 0',
 'qt dec 316 if itw != 2',
 'bi dec -422 if iyu <= -1',
 'uoc inc -862 if itw <= 3',
 'itw dec -301 if x < -632',
 'gif inc -492 if fi != 5',
 'uoc inc -745 if x < -631',
 'xly inc 21 if js > 331',
 'hm inc 44 if js > 334',
 'js dec 503 if tc > 503',
 't inc -216 if j == 396',
 'yg inc 559 if nwe > 189',
 'bhp dec -214 if x >= -646',
 'hm dec 366 if fi == 0',
 't dec -658 if nwe == 185',
 'hm inc -432 if qt <= -307',
 'xly dec 695 if uoc >= -1031',
 'cp inc -438 if x != -647

In [51]:
class VM(object):
    def __init__(self, progl):
        self.prog = progl[:]
        self.registers = dict()
        self.max = 0 # OK because registers initialised to 0
    
    def run(self):
        for l in self.prog:
            self.execute(l)
            m = self.regmax()
            if m > self.max:
                self.max = m
        return self
    
    linematcher = re.compile(r'^([^ ]+) (inc|dec) (-?[0-9]+) if ([^ ]+) ([^ ]+) ([^ ]+)$')

    def execute(self, l):
        target_reg, op, delta, test_l, cond, test_r = VM.linematcher.match(l).groups()
        delta = self.value(delta)
        if op == 'dec':
            delta = -delta
        if self.test(test_l, cond, test_r):
            self.set_reg(target_reg, self.get_reg(target_reg) + delta)   
        return self
    
    def test(self, a, op, b):
        a, b = map(self.value, (a, b))
        if op == '==':
            return a == b
        elif op == '!=':
            return a != b
        elif op == '>':
            return a > b
        elif op == '>=':
            return a >= b
        elif op == '<':
            return a < b
        elif op == '<=':
            return a <= b
        else:
            raise ValueError(f'unimplemented operator {op}')
        

    def value(self, v):
        try:
            return int(v)
        except ValueError:
            return self.get_reg(v)
            
    def get_reg(self, r):
        if r in self.registers:
            return self.registers[r]
        else:
            self.registers[r] = 0
            return 0
        
    def set_reg(self, r, val):
        self.registers[r] = val
    
    def regmax(self):
        try:
            return max(self.registers.values())
        except ValueError:
            return None
    
    def memmax(self):
        return self.max
    
    

In [52]:
class TestIt(unittest.TestCase):
    def test_01(self):
        testVM = VM(test_prog)
        #print('Part 1')
        testVM.run()
        self.assertEqual(testVM.regmax(), 1)
    def test_02(self):
        testVM = VM(test_prog)
        #print('Part 1')
        testVM.run()
        self.assertEqual(testVM.memmax(), 10)

In [53]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestIt)
unittest.TextTestRunner(verbosity=2).run(suite)

test_01 (__main__.TestIt) ... ok
test_02 (__main__.TestIt) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [54]:
vm = VM(real_prog)
vm.run()
print(f'part1 answer {vm.regmax()}\npart 2 answer {vm.memmax()}')

part1 answer 4902
part 2 answer 7037


In [29]:
foo.items()

dict_items([('a', 1), ('b', 2)])

In [31]:
max(foo.values())

2

In [32]:
bar=dict()
max(bar.values())

ValueError: max() arg is an empty sequence

In [38]:
int('aaa')

ValueError: invalid literal for int() with base 10: 'aaa'

In [47]:
10 > None

TypeError: '>' not supported between instances of 'int' and 'NoneType'